In [1]:
import pandas as pd
import os
import numpy as np

import sys
sys.path.append('..')

sys.path.append("../dlplan/experiments")

from sse_and_vf import expand_states, tarski_to_dl_state, episodic_value_iteration
from data_loader import data_loader
from feature_extraction import evaluate_features_at_state
from utils import *
from termcolor import colored

from domain_data import DomainData
from instance_data import InstanceData, state_space_exploration

## On(x,y) task - General Formula

$V(s) = 2n(s) + 2m(s) − E(s) + 2A(s) + 2B(s) − 4D(s) − 2Z(s) + 3$

$A(s) \rightarrow $ block x on y

$B(s) \rightarrow $ block y on x

$E(s) \rightarrow $ gripper is empty

$D(s) \rightarrow $ goal has been achieved

$Z(s) \rightarrow $ whether holding x and y is clear

$n(s) \rightarrow $ # of blocks above x but different than or not above y

$m(s) \rightarrow $ # of blocks above y but different than or not above x




In [3]:
base = '../pddl/blocksworld'

os.listdir()

domain_file = base + '/domain.pddl'
instance_file = base + '/p-3-0-on-linear-simple.pddl'

domain = DomainData(domain_file)
instance = InstanceData(instance_file, domain, 1)

state_space = expand_states(instance, 1000)
V, _, _ = episodic_value_iteration(state_space, instance)

dl_states = [tarski_to_dl_state(instance, state) for state in state_space]

F = construct_syntactic_element_factory(domain)

In [4]:
def compute_value_function(vals, weights):
    assert len(vals) == len(weights), 'should be same lenght'
    return np.dot(weights, vals)
    

In [9]:
# first express is clear or y is on top

block_x = 'c_primitive(on_g,0)'
block_y = 'c_primitive(on_g,1)'


above_x_no_x = f'c_some(r_transitive_closure(r_primitive(on,0,1)),{block_x})'
above_y_inc_y = f'c_some(r_transitive_reflexive_closure(r_primitive(on,0,1)),{block_y})'

above_y_no_y = f'c_some(r_transitive_closure(r_primitive(on,0,1)),{block_y})'
above_x_inc_x = f'c_some(r_transitive_reflexive_closure(r_primitive(on,0,1)),{block_x})'

A_x_on_y = f"b_inclusion(c_primitive(on_g,0),{above_y_no_y})"
B_y_on_x = f"b_inclusion(c_primitive(on_g,1),{above_x_no_x})"

E_arm_empty = 'b_empty(c_primitive(holding,0))'

D_goal = 'n_count(r_and(r_primitive(on,0,1),r_primitive(on_g,0,1)))'    

Z_holding_x_clear_y = f'n_concept_distance(c_primitive(holding,0),r_primitive(on_g,0,1),c_primitive(clear,0))'

N_blocks_above_x_below_y_no_y = f"n_count(c_and({above_x_no_x},c_not({above_y_inc_y})))"
N2_blocks_above_x_below_y_no_y = f"n_count(c_and({above_x_no_x},c_some()))"
N2_blocks_above_x_below_y_no_y = f"{above_x_no_x}"
    
M_blocks_above_y_below_x_no_x = f"n_count(c_and({above_y_no_y},c_not({above_x_inc_x})))"
#M2_blocks_above_y_below_x_no_x = f"n_count(c_and({above_y_no_y},c_not({B_y_on_x})))"
        
# A, _B_, E, X, n, m 

features = [A_x_on_y, 
            B_y_on_x, 
            E_arm_empty,
            D_goal, 
            Z_holding_x_clear_y,
            N_blocks_above_x_below_y_no_y,
            N2_blocks_above_x_below_y_no_y,
            M_blocks_above_y_below_x_no_x,
            #M2_blocks_above_y_below_x_no_x
           ]



In [8]:
# block b3 on b1
names = ['X above Y?', 'Y above X?', 'arm empty?', 'goal achieved?', 
         'Holding X and clear Y?', '# blocks above x?', '# blocks above y?']

for i, state in enumerate(state_space):
    
    dl_state = tarski_to_dl_state(instance, state)
    state = frozenset(state.as_atoms())
    
    vals = evaluate_features_at_state(F, dl_state, features)
    vals[-3] = 0 if vals[-3] > 100 else vals[-3]
    c_v = compute_value_function(vals, [2, 2, -1, -4, -2, 2, 2]) + 3.    
    
    diff = abs(V[state]-(-1 * c_v))
    
    if diff == 0: continue
    
    print(colored(f"State: {str(dl_state)}", "green"))
    print(colored('V(s)', "magenta"), c_v, colored('V*(s)', "red"), abs(V[state]), f"<--> diff={diff}")
    
    for i, j in zip(vals, names):
        print(colored(f"{j} {i}", "blue"))
    print()

RuntimeError: AndConcept::parse_concept_impl - children are not of type Concept.